In [1]:
from tqdm.auto import tqdm
tqdm.pandas()

import wandb
import pandas as pd
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback

from utils import get_predictions, create_iou_table, MIOU, BackgroundIOU, \
                RoadIOU, TrafficLightIOU, TrafficSignIOU, PersonIOU, VehicleIOU, BicycleIOU

In [2]:
class Config:
    WANDB_PROJECT = "mlops-course-001"
    ENTITY = None # set this to team name if working in a team
    BDD_CLASSES = {i: c for i, c in enumerate(['background', 'road', 'traffic light', 'traffic sign',
                                              'person', 'vehicle', 'bicycle'])}
    RAW_DATA_AT = 'bdd_simple_1k'
    PROCESSED_DATA_AT = 'bdd_simple_1k_split'
    
params = Config()

Create a ```train_config``` that we'll pass to W&B ```run``` to control training hyperparameters

In [3]:
train_config = SimpleNamespace(
    framework="fastai",
    img_size=(180, 320),
    batch_size=8,
    augment=True, # use data augmentations
    epochs=10,
    lr=2e-3, 
    pretrained=True, # whether to use a pretrained encoder
    seed=42
)

Set the seed for reproducibility

In [4]:
set_seed(train_config.seed, reproducible=True)

In [5]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type='training', config=train_config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: samu2505. Use `wandb login --relogin` to force relogin


In [6]:
processed_data_at = run.use_artifact(f"{params.PROCESSED_DATA_AT}:latest")
processed_dataset_dir = Path(processed_data_at.download())
df = pd.read_csv(processed_dataset_dir/'data_split.csv')

wandb: Downloading large artifact bdd_simple_1k_split:latest, 846.07MB. 4010 files... Done. 0:0:0.4


In [7]:
df = df[df.stage != 'test'].reset_index(drop=True)
df['is_valid'] = df.stage =='valid'
df

,File_Name,stage,is_valid
0,4b4cb9b0-3efa6143.jpg,valid,True
1,5715cd38-af2ede7c.jpg,train,False
2,6d805841-54db0849.jpg,train,False
3,b6b047b4-3fccc761.jpg,train,False
4,8c976e04-47482559.jpg,train,False
...,...,...,...
895,68e7781a-cffc2268.jpg,train,False
896,3b87b80e-d856ff82.jpg,train,False
897,350334d0-07862427.jpg,train,False
898,351bec53-3749220c.jpg,train,False


In [8]:
def label_func(fname):
    return (fname.parent.parent/'labels')/f"{fname.stem}_mask.png"

In [9]:
# assign paths
df['image_fname'] = [processed_dataset_dir/f'images/{f}' for f in tqdm(df.File_Name.values, total=len(df),
                                                                          desc='Creating image files')]
df['label_fname'] = [label_func(f) for f in tqdm(df.image_fname.values, total=len(df),
                                                desc='Creating labels')]

Creating image files:   0%|          | 0/900 [00:00<?, ?it/s]

Creating labels:   0%|          | 0/900 [00:00<?, ?it/s]

In [10]:
df

,File_Name,stage,is_valid,image_fname,label_fname
0,4b4cb9b0-3efa6143.jpg,valid,True,artifacts/bdd_simple_1k_split:v3/images/4b4cb9b0-3efa6143.jpg,artifacts/bdd_simple_1k_split:v3/labels/4b4cb9b0-3efa6143_mask.png
1,5715cd38-af2ede7c.jpg,train,False,artifacts/bdd_simple_1k_split:v3/images/5715cd38-af2ede7c.jpg,artifacts/bdd_simple_1k_split:v3/labels/5715cd38-af2ede7c_mask.png
2,6d805841-54db0849.jpg,train,False,artifacts/bdd_simple_1k_split:v3/images/6d805841-54db0849.jpg,artifacts/bdd_simple_1k_split:v3/labels/6d805841-54db0849_mask.png
3,b6b047b4-3fccc761.jpg,train,False,artifacts/bdd_simple_1k_split:v3/images/b6b047b4-3fccc761.jpg,artifacts/bdd_simple_1k_split:v3/labels/b6b047b4-3fccc761_mask.png
4,8c976e04-47482559.jpg,train,False,artifacts/bdd_simple_1k_split:v3/images/8c976e04-47482559.jpg,artifacts/bdd_simple_1k_split:v3/labels/8c976e04-47482559_mask.png
...,...,...,...,...,...
895,68e7781a-cffc2268.jpg,train,False,artifacts/bdd_simple_1k_split:v3/images/68e7781a-cffc2268.jpg,artifacts/bdd_simple_1k_split:v3/labels/68e7781a-cffc2268_mask.png
896,3b87b80e-d856ff82.jpg,train,False,artifacts/bdd_simple_1k_split:v3/images/3b87b80e-d856ff82.jpg,artifacts/bdd_simple_1k_split:v3/labels/3b87b80e-d856ff82_mask.png
897,350334d0-07862427.jpg,train,False,artifacts/bdd_simple_1k_split:v3/images/350334d0-07862427.jpg,artifacts/bdd_simple_1k_split:v3/labels/350334d0-07862427_mask.png
898,351bec53-3749220c.jpg,train,False,artifacts/bdd_simple_1k_split:v3/images/351bec53-3749220c.jpg,artifacts/bdd_simple_1k_split:v3/labels/351bec53-3749220c_mask.png


In [11]:
def get_data(df, bs=4, img_size=(180, 320), augment=True):
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=params.BDD_CLASSES)),
                     get_x=ColReader('image_fname'),
                     get_y=ColReader('label_fname'),
                     splitter=ColSplitter(),
                     item_tfms=Resize(img_size),
                     batch_tfms=aug_transforms() if augment else None)
    
    return block.dataloaders(df, bs=bs)

In [12]:
config = wandb.config

In [13]:
dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/_tensor.py:1051: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ret = func(*args, **kwargs)


In [14]:
metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(), \
          TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]

learn = unet_learner(dls, arch=resnet18, pretrained=config.pretrained, metrics=metrics)

In [16]:
callbacks = [
    SaveModelCallback(monitor='miou'),
    WandbCallback(log_preds=False, log_model=True)
]

callbacks

[SaveModelCallback, WandbCallback]

In [18]:
learn.fit_one_cycle(config.epochs, config.lr, cbs=callbacks)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/_tensor.py:1051: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ret = func(*args, **kwargs)


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.490677,0.349959,0.297394,0.857789,0.676493,0.000000,0.000000,0.000000,0.547474,0.000000,00:41
1,0.466264,0.553690,0.222798,0.789950,0.315442,0.000000,0.000000,0.000000,0.454197,0.000000,00:41
2,0.367835,0.405159,0.318156,0.875586,0.745007,0.000000,0.000000,0.000000,0.606501,0.000000,00:40
3,0.321802,0.314485,0.321257,0.872683,0.711619,0.000000,0.000000,0.000000,0.664494,0.000000,00:40
4,0.281379,0.251811,0.343938,0.903793,0.790577,0.000440,0.000000,0.000000,0.712753,0.000000,00:40
5,0.245501,0.247902,0.338116,0.898746,0.800793,0.000000,0.000000,0.000000,0.667272,0.000000,00:40
6,0.235961,0.250313,0.353900,0.910253,0.794903,0.048807,0.000000,0.000000,0.723334,0.000000,00:40
7,0.211294,0.244943,0.351020,0.910322,0.801669,0.009864,0.000000,0.000000,0.735283,0.000000,00:40
8,0.193634,0.228679,0.359112,0.915457,0.813365,0.032538,0.000760,0.000000,0.751661,0.000000,00:40
9,0.187169,0.230073,0.362017,0.915586,0.814348,0.047882,0.004183,0.000000,0.752119,0.000000,00:41


Better model found at epoch 0 with miou value: 0.2973936962635032.
Better model found at epoch 2 with miou value: 0.318156267917528.
Better model found at epoch 3 with miou value: 0.3212565714067304.
Better model found at epoch 4 with miou value: 0.3439375260325666.
Better model found at epoch 6 with miou value: 0.353899672661179.
Better model found at epoch 8 with miou value: 0.35911159512961305.
Better model found at epoch 9 with miou value: 0.36201670081194653.


Log a table with model predictions and ground truth to W&B, so that we can do error analysis in the W&B dashboard

In [19]:
samples, outputs, predictions = get_predictions(learn)
table = create_iou_table(samples, outputs, predictions, params.BDD_CLASSES)
wandb.log({'pred_table': table})

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/_tensor.py:1051: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ret = func(*args, **kwargs)


We're reloading the model from the best checkpoint at the end and saving it. To make sure we track the final metrics correctly, we'll validate the model again and save the final loss and metrics to ```wandb.summary```

In [20]:
scores = learn.validate()
metric_names = ['final_loss'] + [f'final_{x.name}' for x in metrics]
final_results = {metric_names[i]: scores[i] for i in range(len(scores))}

for k, v in final_results.items():
    wandb.summary[k] = v

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/_tensor.py:1051: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ret = func(*args, **kwargs)


In [21]:
wandb.finish()

background_iou,▅▁▆▆▇▇████
bicycle_iou,▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_1,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_2,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
miou,▅▁▆▆▇▇█▇██
mom_0,██▇▆▅▄▃▂▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██████
